<h2> 異分野共著　process II </h2>

<h3> 4. 全著者IDそれぞれに専門分野を割り当てる </h3>

In [22]:
#! Python3
# -*- coding: utf-8 -*-
import pandas as pd
import requests
import xml.etree.ElementTree as ET
import urllib
from urllib.error import URLError, HTTPError

# API Key   
#myAPIkey = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
myAPIkey = "36296ac3d13667efc3724906103f9bbc"
myInsttoken = 'd344993c91b6e9daf0b5c734a06463aa'

In [23]:
# 調べたい Author 
uniqueaIds = pd.read_csv('UniqueAuthorIdsList.csv', dtype= 'object')
uniqueaIds['ID'] = "'" + uniqueaIds['Author ID'] + "'"


# APIのひな型
#api =  'https://api.elsevier.com/content/author/author_id/{sid}?apiKey={key}'
api =  'https://api.elsevier.com/content/author/author_id/{sid}?apiKey={key}&insttoken={key2}'


author_asjc27 = []
#4-1. 著者IDから分野（ASJC, 27分野） codeを抽出（API）
for i, number in uniqueaIds.iterrows():
    # APIのURLを得る
    aid = number['Author ID']
    print("AID=",aid)
#        url = api.format(sid=aid, key=myAPIkey)
    url = api.format(sid=aid, key=myAPIkey, key2=myInsttoken)

    print(url)
    print('Scopus Author ID'+ str(i) + ' : ' + str(aid) )

    # 実際にAPIにリクエストを送信して結果を取得する
    try:
        req = urllib.request.Request(url)
        with urllib.request.urlopen(req) as response:
            r = requests.get(url)
    except URLError as e:
        author27 = [aid, 0, 0, 0]
        print(author27)
        author_asjc27.append(author27)
        if hasattr(e, 'reason'):
            print('Reason: ', e.reason)
            #APIkeyの制限超えたらリストを保存して止まる
            if e.reason == 'Too Many Requests':
                break
        elif hasattr(e, 'code'):
            print('Error code: ', e.code)
    else:
        text = r.text.encode('utf-8')
        data = ET.fromstring(text)

        # 結果を出力 (Author id, subject-areas)
        #  Defined Author ID
        author_id = data[0][1].text.replace('AUTHOR_ID:','')

        # 論文数が5本以下ならスキップ
        try:
            dc = int(data.find('.//document-count').text)
        except:
            dc = 0

        if dc < 5:
            asjc27 = 0
            author27 = [aid, author_id, dc, asjc27]
        else:
            #author-profileの下から、分野のコードcode(4桁のうち上位2桁)とその頻度frequencyを抽出
            asjc = []
            asjc_list = []
            for et in data.findall('.//author-profile/classificationgroup/classifications[@type="ASJC"]/classification'):
                code = int(et.text[:2])
                freqs = int(et.get('frequency'))
                asjc = [code, freqs]
                asjc_list.append(asjc)

            #4-2. 一番多い分野を特定, 著者の分野を決定
            asjcdf = pd.DataFrame(asjc_list, columns=['code2', 'freqs'])
            asjcsort = asjcdf.groupby('code2',as_index=False)['freqs'].sum()
            asjcsort.sort_values(by='freqs', ascending=False, inplace = True)
            asjc27 = asjcsort.iat[0,0]
            author27 = [aid, author_id, dc, asjc27]
        print(author27)
        author_asjc27.append(author27)
    
#4-4. listを作成
author_asjc27_df = pd.DataFrame(author_asjc27, columns=['Scopus Author ID', 'responce ID', 'document count', 'asjc27'])
author_asjc27_df.to_csv('AuthorIDwithasjc27List.csv', index = False)


AID= 55553077000
https://api.elsevier.com/content/author/author_id/55553077000?apiKey=36296ac3d13667efc3724906103f9bbc&insttoken=d344993c91b6e9daf0b5c734a06463aa
Scopus Author ID0 : 55553077000
['55553077000', '55553077000', 41, 27]
AID= 57213186397
https://api.elsevier.com/content/author/author_id/57213186397?apiKey=36296ac3d13667efc3724906103f9bbc&insttoken=d344993c91b6e9daf0b5c734a06463aa
Scopus Author ID1 : 57213186397
['57213186397', '57213186397', 6, 27]
AID= 57160212700
https://api.elsevier.com/content/author/author_id/57160212700?apiKey=36296ac3d13667efc3724906103f9bbc&insttoken=d344993c91b6e9daf0b5c734a06463aa
Scopus Author ID2 : 57160212700
['57160212700', '57160212700', 24, 27]
AID= 9845935500
https://api.elsevier.com/content/author/author_id/9845935500?apiKey=36296ac3d13667efc3724906103f9bbc&insttoken=d344993c91b6e9daf0b5c734a06463aa
Scopus Author ID3 : 9845935500


KeyboardInterrupt: 